In [1]:
import flexmatcher
import pandas as pd
from valentine import valentine_match
from valentine.algorithms import Coma, Cupid, DistributionBased, JaccardLevenMatcher, SimilarityFlooding
import pickle
import nltk
nltk.download('stopwords')

/home/edkl/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to /home/edkl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
data_folder = "../crawl/crawl/spiders/all_data"

In [3]:
okxe_data = pd.read_csv(data_folder + "/okxe.csv").drop(columns=['Unnamed: 0'], axis=1)
muaban_data = pd.read_csv(data_folder + "/muaban.csv").drop(columns=['Unnamed: 0'], axis=1)
webike_data = pd.read_csv(data_folder + "/webike.csv").drop(columns=['Unnamed: 0'], axis=1)
xemayhoangkien_data = pd.read_csv(data_folder + "/xemayhoangkien.csv").drop(columns=['Unnamed: 0'], axis=1)

In [4]:
muaban_data.head()

,name,price,address,url,Hãng xe,Năm đăng ký,Tình trạng,Loại xe máy,Dung tích xe máy,Xuất xứ,Màu xe,Dòng xe,Số km đã đi
0,CẦN BÁN XE MÁY HONDA VISION 110 ĐỜI 2022,15000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2022.0,Mới,Tay ga,101 - 175 cc,Nước khác,Đen,-,0
1,"Chuyên thu mua các loại xe máy cũ và mới, xe h...",0,"Quận Đống Đa, Hà Nội",https://muaban.net/xe-may/mua-xe-may/chuyen-th...,-,-,-,-,-,-,-,-,0
2,HEAD SƠN TÙNG 1 - TRỢ GIÁ MUA XE TƯNG BỪNG-VUI...,31990000,"Huyện Hoài Đức, Hà Nội",https://muaban.net/xe-may/ban-xe-may/head-son-...,Honda,-,Mới,Tay côn/Moto,-,-,-,-,0
3,Cần thanh lý lô xe honda AirBlade 150 nhập khẩu,19000000,"TP. Thủ Đức - Quận 9, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-thanh...,Honda,2022.0,Mới,Tay ga,101 - 175 cc,Nước khác,Đen,Air Blade,0
4,CẦN BÁN XE MÁY HONDA SH150 ĐỜI 2022,32000000,"Quận 5, TP.HCM",https://muaban.net/xe-may/ban-xe-may/can-ban-x...,Honda,2023.0,Mới,Tay ga,101 - 175 cc,Nước khác,Ghi/Xám,SH,0


In [5]:
webike_data.head()

,name,price,address,url,Dòng xe,Đã đi,Xuất xứ,Đời xe,Phân khối,Loại xe,Màu,Ngày đăng
0,Cần bán PIAGGIO Liberty 3V i.e 125 2015 màu trắng,19500000.0,"Cầu Giấy , Hà Nội",https://www.webike.vn/cho-xe-may/bike-detail/c...,PIAGGIO Liberty 3V i.e 125 2015,10 000 - 14 999 km,Việt Nam,2015,125 cc,Xe tay ga,Trắng,11 ngày trước
1,HONDA Airblade 110 Fi Đen Đỏ Trắng - 2013,10500000.0,"Cầu Giấy , Hà Nội",https://www.webike.vn/cho-xe-may/bike-detail/h...,HONDA Airblade 110 Fi 2013,5 000 - 9 999 km,Nhật Bản,2013,110 cc,Xe tay ga,Đen Đỏ Trắng,16 ngày trước
2,Cần bán HONDA SH 150i 2014 màu trắng,60000000.0,"An Khê , Gia Lai",https://www.webike.vn/cho-xe-may/bike-detail/c...,HONDA SH 150i 2014,40 000 - 44 999 km,Việt Nam,2014,150 cc,Xe tay ga,Trắng,5 ngày trước
3,Cần bán PIAGGIO Vespa S 3V i.e 125 2016 màu xa...,0.0,"Cầu Giấy , Hà Nội",https://www.webike.vn/cho-xe-may/bike-detail/c...,PIAGGIO Vespa S 3V i.e 125 2016,10 000 - 14 999 km,Việt Nam,2016,125 cc,Xe tay ga,Xanh Da Trời Xám,1 tháng trước
4,HONDA SH Mode 125 Đen Đỏ - 2019,47000000.0,"Tân Phú , Hồ Chí Minh",https://www.webike.vn/cho-xe-may/bike-detail/h...,HONDA SH Mode 125 2019,15 000 - 19 999 km,Việt Nam,2019,125 cc,Xe tay ga,Đen Đỏ,16 ngày trước


In [6]:
xemayhoangkien_data.head()

,name,price,url,Thương hiệu,Loại,MSP,Năm đăng ký,Dung tích,Màu sắc
0,Winner X ABS 22Y-222BG,39500000,https://xemayhoangkien.com/winner-x-abs-29y-222bg,Honda,Winner,-,-,-,-
1,LX 30L-48DC,14200000,https://xemayhoangkien.com/lx-30l-48dc,Piaggio,LX,-,-,-,-
2,Wave A 38K-545CB,15800000,https://xemayhoangkien.com/wave-a-38k-545cb,Honda,Wave A,38K-545CB,2020,110cc,Trắng đen bạc
3,LX ie 29X-113JK,20000000,https://xemayhoangkien.com/lx-ie-29x-113jk,Piaggio,LX,29X-113JK,2013 (đời 2012),125cc,Vàng
4,TFX 89B-465GG,39300000,https://xemayhoangkien.com/tfx-89b-465gg,Yamaha,TFX,89B-465GG,2017,150cc,Đỏ đen


In [7]:
okxe_data.head()

,name,price,address,url,brand,image_url,model,store,type,fuel,...,used_distance,Giảm xóc - Phuộc,Phanh,Động cơ,Đồng hồ,Khoá,Hệ thống đèn,Hệ thống tự động,Công nghệ kết nối,Khác
0,Suzuki GSX 150 Bandit,28000000,Tp. Hồ Chí Minh,https://api.okxe.vn/products/suzuki-gsx-150-br...,Suzuki,https://cdn.okxe.vn/products/848/1689848/16898...,GSX,Trạm Dịch Vụ Xe Máy TP HCM,2,Gasolin,...,0-10600km,Phuộc Showa SFF-CA & monoshock Showa BFRC,Phanh Đĩa,Phun xăng điện tử Fi / PGM-FI,Đồng hồ LCD,-,-,-,-,-
1,Honda Air Blade 160 FI,58000000,Tp. Hà Nội,https://api.okxe.vn/products/honda-air-blade-1...,Honda,https://cdn.okxe.vn/products/503/1690503/16905...,Air Blade,Trạm Dịch Vụ Xe Máy OKXE Hà Nội,1,Gasolin,...,0-3000km,-,"Phanh ABS,Phanh Đĩa",Phun xăng điện tử Fi / PGM-FI,Đồng hồ điện tử,"Smartkey,Khóa chống trộm",Đèn LED,Hệ thống ngắt động cơ tạm thời - Idling Stop,USB post,"Tiết kiệm nhiên liệu,Hiển thị Eco"
2,Honda Air Blade 125 FI,35000000,Tp. Hồ Chí Minh,https://api.okxe.vn/products/honda-air-blade-1...,Honda,https://cdn.okxe.vn/products/241/1683241/16832...,Air Blade,Trạm Dịch Vụ Xe Máy TP HCM,1,Gasolin,...,0-7000km,-,"Phanh cơ,Phanh Đĩa",Phun xăng điện tử Fi / PGM-FI,-,-,-,Hệ thống ngắt động cơ tạm thời - Idling Stop,-,-
3,Honda Air Blade 125 FI,30500000,Tp. Hồ Chí Minh,https://api.okxe.vn/products/honda-air-blade-1...,Honda,https://cdn.okxe.vn/products/822/1689822/16898...,Air Blade,Trạm Dịch Vụ Xe Máy TP HCM,1,Gasolin,...,0-60000km,-,"Phanh cơ,Phanh Đĩa",Phun xăng điện tử Fi / PGM-FI,-,-,-,Hệ thống ngắt động cơ tạm thời - Idling Stop,-,-
4,Honda SH 125i,60000000,Tp. Hồ Chí Minh,https://api.okxe.vn/products/honda-sh-125i-168...,Honda,https://cdn.okxe.vn/products/740/1689740/16897...,SH,Trạm Dịch Vụ Xe Máy TP HCM,1,Gasolin,...,0-5000km,-,-,Phun xăng điện tử Fi / PGM-FI,Đồng hồ LCD,-,-,Hệ thống ngắt động cơ tạm thời - Idling Stop,-,-


In [8]:
matcher = Coma(strategy="COMA_OPT_INST")
matches = valentine_match(muaban_data, okxe_data, matcher, 'muaban', 'okxe')

In [9]:
matches

{(('muaban', 'price'), ('okxe', 'price')): 0.7328221,
 (('muaban', 'name'), ('okxe', 'name')): 0.66536885,
 (('muaban', 'address'), ('okxe', 'address')): 0.6599225,
 (('muaban', 'url'), ('okxe', 'url')): 0.62242526,
 (('muaban', 'Tình trạng'), ('okxe', 'used_status')): 0.3341773,
 (('muaban', 'Hãng xe'), ('okxe', 'brand')): 0.3338325}

In [10]:
matcher = Coma(strategy="COMA_OPT_INST")
matches = valentine_match(muaban_data, webike_data, matcher, 'muaban', 'webike')

In [11]:
matches

{(('muaban', 'address'), ('webike', 'address')): 0.79283416,
 (('muaban', 'Dòng xe'), ('webike', 'Dòng xe')): 0.77932745,
 (('muaban', 'Xuất xứ'), ('webike', 'Xuất xứ')): 0.7542795,
 (('muaban', 'name'), ('webike', 'name')): 0.75102365,
 (('muaban', 'url'), ('webike', 'url')): 0.72343534,
 (('muaban', 'price'), ('webike', 'price')): 0.67715883,
 (('muaban', 'Loại xe máy'), ('webike', 'Loại xe')): 0.64975417,
 (('muaban', 'Màu xe'), ('webike', 'Màu')): 0.52360696,
 (('muaban', 'Số km đã đi'), ('webike', 'Đã đi')): 0.49733377,
 (('muaban', 'Năm đăng ký'), ('webike', 'Ngày đăng')): 0.3750846}

In [12]:
matcher = Coma(strategy="COMA_OPT_INST")
matches = valentine_match(muaban_data, xemayhoangkien_data, matcher, 'muaban', 'xemayhoangkien')

In [13]:
matches

{(('muaban', 'price'), ('xemayhoangkien', 'price')): 0.758012,
 (('muaban', 'name'), ('xemayhoangkien', 'name')): 0.6697773,
 (('muaban', 'url'), ('xemayhoangkien', 'url')): 0.64991707,
 (('muaban', 'Năm đăng ký'), ('xemayhoangkien', 'Năm đăng ký')): 0.6469589,
 (('muaban', 'Màu xe'), ('xemayhoangkien', 'Màu sắc')): 0.5456339,
 (('muaban', 'Dung tích xe máy'), ('xemayhoangkien', 'Dung tích')): 0.46834582,
 (('muaban', 'Hãng xe'), ('xemayhoangkien', 'Thương hiệu')): 0.4044152,
 (('muaban', 'Loại xe máy'), ('xemayhoangkien', 'Loại')): 0.380536}

In [14]:
def read_data(data_path: str):
    # mediated_schema = pd.read_csv(data_path + '/mediated_schema.csv')
    okxe_data = pd.read_csv(data_folder + "/okxe.csv").drop(columns=['Unnamed: 0'], axis=1)
    muaban_data = pd.read_csv(data_folder + "/muaban.csv").drop(columns=['Unnamed: 0'], axis=1)
    webike_data = pd.read_csv(data_folder + "/webike.csv").drop(columns=['Unnamed: 0'], axis=1)
    xemayhoangkien_data = pd.read_csv(data_folder + "/xemayhoangkien.csv").drop(columns=['Unnamed: 0'], axis=1)
    return okxe_data, muaban_data, webike_data, xemayhoangkien_data


In [15]:
def schema_matching(matcher, config_folder: str):
    okxe_mediated = valentine_match(muaban_data, okxe_data, matcher, 'muaban', 'okxe')
    xemayhoangkien_mediated = valentine_match(muaban_data, xemayhoangkien_data, matcher, 'muaban', 'xemayhoangkien')
    webike_mediated = valentine_match(muaban_data, webike_data, matcher, 'muaban', 'webike')

    for result in [okxe_mediated, xemayhoangkien_mediated, webike_mediated]:
        file_name = None
        mapping = {}
        for key, value in result.items():
            file_name = key[0][0] + '_' + key[1][0]
            attr1, attr2 = key[0][1], key[1][1]
            mapping[attr1] = attr2
        with open(config_folder + '/' + file_name + '.pkl', 'wb') as file:
            pickle.dump(mapping, file)


In [25]:
matcher = Coma(strategy="COMA_OPT_INST")
config_mapping = '/home/edkl/project/20222/THDL/BTL/THDL/config_mapping'
schema_matching(matcher=matcher, config_folder=config_mapping)